In [1]:
import pandas as pd
import numpy as np

In [3]:
%%sh
head ./data/data.csv

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850,United Kingdom


In [121]:
df = pd.read_csv('./data/data.csv', 
                 encoding='ISO-8859-1',
                 dtype={'CustomerID':str},
                 parse_dates=['InvoiceDate'])

In [122]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [151]:
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day

In [152]:
final = df.loc[((df['Year']==2011) & (df['Month']<=11)) | 
               ((df['Year']==2010) & (df['Month']==12)),
                 ['StockCode','Quantity','Month']]
#unit-price, Description
#rearrange 12 to 0 as it belongs to 2011
final.loc[final.Month==12,:]=0
final = final[final.Quantity>0]
data_set = final.groupby(by=['StockCode','Month']).sum().reset_index().pivot(
    index='StockCode',
    columns='Month',
    values="Quantity").fillna(0)

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
estimator = KMeans(n_clusters=5)

estimator.fit(data_set) #scaler.fit_transform(data_set) #(data_set.T/data_set.sum(axis=1)).T
pd.Series(estimator.labels_).value_counts()

0    3283
3     456
4      84
2       8
1       1
dtype: int64

In [156]:
estimator.cluster_centers_

final = df.loc[((df['Year']==2011) & (df['Month']<=11)) | 
               ((df['Year']==2010) & (df['Month']==12)),
                 ['StockCode','Quantity','Month']]
final.loc[final.Month==12,'Month'] = 0
final.head()

,StockCode,Quantity,Month
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [146]:
data_set.to_csv('product_prices_12m.csv',index=False)

df[['StockCode','Description','UnitPrice']].to_csv('product_meta_data.csv',index=False)

In [143]:
%%sh 
head product_meta_data.csv

StockCode,Description,UnitPrice
85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55
71053,WHITE METAL LANTERN,3.39
84406B,CREAM CUPID HEARTS COAT HANGER,2.75
84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39
84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39
22752,SET 7 BABUSHKA NESTING BOXES,7.65
21730,GLASS STAR FROSTED T-LIGHT HOLDER,4.25
22633,HAND WARMER UNION JACK,1.85
22632,HAND WARMER RED POLKA DOT,1.85


In [147]:
%%sh
head product_prices_12m.csv

1,2,3,4,5,6,7,8,9,10,11
340.0,52.0,208.0,189.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,2.0,0.0,0.0,0.0,60.0,194.0,60.0,60.0,6.0,113.0
0.0,30.0,29.0,0.0,3.0,0.0,5.0,35.0,10.0,10.0,49.0
0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
213.0,0.0,114.0,82.0,107.0,15.0,177.0,85.0,210.0,51.0,62.0
180.0,39.0,83.0,76.0,33.0,526.0,830.0,763.0,207.0,0.0,0.0
613.0,272.0,118.0,30.0,13.0,55.0,177.0,152.0,70.0,70.0,181.0
